In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import cv2
import glob

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [4]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import models,layers
from tensorflow.keras.utils import to_categorical


In [5]:
from joblib import dump

In [6]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")


#Add Dataset

In [7]:
%%capture
!pip install kaggle

In [8]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/DeepLearning/CatDog/Kaggle'

In [9]:
%cd /content/drive/MyDrive/DeepLearning/CatDog/Kaggle

/content/drive/MyDrive/DeepLearning/CatDog/Kaggle


In [10]:
!pwd

/content/drive/MyDrive/DeepLearning/CatDog/Kaggle


In [11]:
!kaggle datasets download -d tongpython/cat-and-dog

cat-and-dog.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
%%capture
!unzip /content/drive/MyDrive/DeepLearning/CatDog/Kaggle/cat-and-dog.zip -d /content/drive/MyDrive/DeepLearning/CatDog/Dataset

#KNN

In [ ]:
feature_vectors = []
labels = []

In [ ]:
def get_label(address):
  label = address.split("/")[-2]
  labels.append(label)

In [ ]:
#Import Dataset
for i,address in enumerate(glob.glob('/content/drive/MyDrive/DeepLearning/CatDog/Dataset/training_set/training_set/*/**')):
  #Read Image
  img = cv2.imread(address)
  # Check if the image is loaded successfully
  if img is not None:
      # Resize image
      img = cv2.resize(img ,(32,32))
      # Normalize image
      img = img/255.0
      #Flat
      img = img.flatten()
      #append
      feature_vectors.append(img)
      #Give label
      get_label(address)
  else:
      print("Failed to load the image:", address)

  if i % 100 == 0 :
    print(f"Info :{i+1} processed")
print(f"Finished")

In [ ]:
#Convert to numpy
np_feature_vectors = np.array(feature_vectors)

In [ ]:
#Split
x_train, x_test, y_train, y_test =train_test_split(np_feature_vectors, labels, test_size=0.2,random_state=42)

In [ ]:
#Model
model = KNeighborsClassifier()
model.fit(x_train, y_train)

In [ ]:
#Prediction
accuracy = model.score(x_test, y_test)
accuracy

In [ ]:
#Prediction
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

#MLP

In [ ]:
column = x_train.shape[1]
column

In [ ]:

# Assuming 'target' is the name of the target variable column in your dataset
target = labels

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Fit the label encoder to the target variable
label_encoder.fit(target)

# Transform the target variable using label encoding
encoded_labels = label_encoder.transform(target)

In [ ]:

encoded_labels

In [ ]:
categ_labels = to_categorical(encoded_labels)

In [ ]:
categ_labels

In [ ]:
#Split
x_train, x_test, y_train, y_test =train_test_split(np_feature_vectors, categ_labels, test_size=0.2,random_state=42)


In [ ]:
#Model
model = models.Sequential()
model.add(layers.Dense(700, activation = 'relu',input_dim =column))
model.add(layers.Dense(100, activation = 'relu'))
model.add(layers.Dense(50, activation = 'relu'))
model.add(layers.Dense(25, activation = 'relu'))
model.add(layers.Dense(2, activation = 'softmax'))


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])


In [ ]:
model.summary()


In [ ]:
result = model.fit(x_train, y_train, validation_data =(x_test, y_test ), epochs= 20 ,batch_size=32)

In [ ]:
pd.DataFrame(result.history).plot()
